In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
with open('matches.json', 'r') as test_js:
    j_data = test_js.read() 

In [3]:
df = pd.read_json(j_data)

In [4]:
#CHECK NUMBER OF FEATURES IN EACH COLUMN
for col_name in df.columns:
    if df[col_name].dtype == 'object':
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(
            col_name=col_name, unique_cat=unique_cat))

Feature 'country' has 168 unique categories
Feature 'tournament' has 1096 unique categories
Feature 'host' has 15986 unique categories
Feature 'guest' has 15018 unique categories
Feature 'ht_score' has 75 unique categories
Feature 'ft_score' has 169 unique categories


In [5]:
#CREATE NEW COLUMN WITH SCORE DIFFRENCE IN HALF TIME
df['ht score difference'] = [abs(int(x.split(':')[0]) - int(x.split(':')[1])) for x in df['ht_score']]

In [6]:
df.head()

,year,month,day,country,tournament,host,guest,ht_score,ft_score,broken,ht score difference
0,2019,2,26,ALGERIA,ALGERIA: LIGUE 2,Harrach,RC Kouba,0:0,0:2,0,0
1,2019,2,26,ANGOLA,ANGOLA: GIRABOLA,Kabuscorp,Cubango,1:0,3:2,0,1
2,2019,2,26,ARGENTINA,ARGENTINA: SUPERLIGA,Newells Old Boys,San Martin S.J.,1:0,3:0,0,1
3,2019,2,26,ARGENTINA,ARGENTINA: PRIMERA B NACIONAL,Temperley,Ferro,0:1,2:2,0,1
4,2019,2,26,ARGENTINA,ARGENTINA: PRIMERA B METROPOLITANA,Barracas Central,Dep. Riestra,1:0,1:0,0,1


In [7]:
#PREPARE GROUPA MADE FROM COUNTRIES -> DIVIDE ALL COUNTRIES BY CONTINENT
eu_countries = ['RUSSIA', 'GERMANY', 'ENGLAND', 'FRANCE', 'ITALY', 'SPAIN', 'UKRAINE', 'POLAND', 'ROMANIA',
                'NETHERLANDS', 'BELGIUM', 'GREECE', 'CZECH REPUBLIC', 'PORTUGAL', 'SWEDEN', 'HUNGARY', 'BELARUS',
                'AUSTRIA', 'SERBIA', 'SWITZERLAND', 'BULGARIA', 'DENMARK', 'FINLAND', 'SLOVAKIA', 'NORWAY', 'IRELAND',
                'CROATIA', 'MOLDOVA', 'BOSNIA AND HERZEGOVINA', 'ALBANIA', 'LITHUANIA', 'SLOVENIA', 'LATVIA', 'ESTONIA',
                'MONTENEGRO', 'LUXEMBOURG', 'MALTA', 'ICELAND', 'ANDORRA', 'MONACO', 'LIECHTENSTEIN', 'SAN MARINO',
                'GIBRALTAR', 'NORTHERN IRELAND', 'SCOTLAND', 'NORTH MACEDONIA', 'KOSOVO', 'WALES', 'FAROE ISLANDS']
eu_tournament = ['EUROPE']
asia_countries = ['MALDIVES', 'MYANMAR', 'PALESTINE', 'CHINA', 'INDIA', 'INDONESIA', 'PAKISTAN', 'BANGLADESH', 'JAPAN', 
                  'PHILIPPINES', 'VIETNAM', 'TURKEY',
                  'IRAN', 'THAILAND', 'MAYANMAR', 'SOUTH KOREA', 'IRAQ', 'AFGHANISTAN', 'SAUDI ARABIA', 'UZBEKISTAN', 
                 'MALAYSIA', 'NEPAL', 'YEMEN', 'NORTH KOREA', 'SRI LANKA', 'KAZAKHSTAN', 'SYRIA', 'CAMBODIA', 'JORDAN', 
                 'AZERBAIJAN', 'UNITED ARAB EMIRATES', 'TAJIKISTAN', 'ISRAEL', 'LAOS', 'KYRGYZSTAN', 'LEBANON', 
                 'TURKMENISTAN', 'SINGAPORE', 'OMAN', 'KUWAIT', 'GEORGIA', 'MONGOLIA', 'ARMENIA', 'QATAR', 'BAHRAIN', 
                 'CYPRUS', 'BHUTAN', 'MALEDIVES', 'BRUNEI', 'HONG KONG']
asia_tournament = ['ASIA']
n_america_countries = ['USA', 'BERMUDA', 'MEXICO', 'CANADA', 'GUATEMALA', 'CUBA', 'HAITI', 'DOMINICAN REPUBLIC', 
                       'HONDURAS', 'EL SALVADOR', 'NICARAGUA', 'COSTA RICA', 'PANAMA', 'PUERTO RICO', 'JAMAICA', 
                       'TRINIDAD AND TOBAGO', 'GUADELOUPE', 'MARTINIQUE', 'BAHAMAS', 'BELIZE', 'BARBADOS', 
                       'SAINT LUCIA']
north_america_tournament = ['NORTH & CENTRAL AMERICA']
s_america_countries = ['BRAZIL', 'COLOMBIA', 'ARGENTINA', 'PERU', 'VENEZUELA', 'CHILE', 'ECUADOR', 'BOLIVIA', 'PARAGUAY', 
                      'URUGUAY', 'GUYANA']
south_america_tournament = ['SOUTH AMERICA']
africa_countries = ['CAPE VERDE', 'RÉUNION', 'ESWATINI', 'REPUBLIC OF THE CONGO', 'NAMIBIA', 'MAURITIUS', 'IVORY COAST', 
                    'NIGERIA', 'ETHIOPIA', 'EGYPT', 'DR CONGO', 'TANZANIA', 'SOUTH AFRICA', 'KENYA', 'UGANDA', 'ALGERIA', 
                   'SUDAN', 'MOROCCO', 'ANGOLA', 'MOZAMBIQUE', 'GHANA', 'MADAGASCAR', 'CAMEROON', 'NIGER', 'BURKINA FASO', 
                   'MALAWI', 'MALI', 'ZAMBIA', 'ZIMBABWE', 'SENEGAL', 'CHAD', 'SOMALIA', 'GUINEA', 'SOUTH SUDAN', 'RWANDA', 
                   'BENIN', 'TUNISIA', 'BURUNDI', 'TOGO', 'SIERRA LEONE', 'LIBYA', 'CONGO', 'ERITREA', 'LIBERIA', 
                    'MAURITANIA', 'NAMBIA', 'BOTSWANA', 'LESOTHO', 'GAMBIA', 'GABON', 'DJIBOUTI', 'COMOROS', 'SEYCHELLES']
africa_tournament = ['AFRICA']
australia_countries = ['AUSTRALIA & OCEANIA', 'NEW ZEALAND', 'PAPUA NEW GUINEA', 'FIJI', 'MARSHALL ISLANDS',
                       'PALAU']
australia_tournament = ['AUSTRALIA']
world = ['WORLD']
world_club_friendly = ['WORLD: CLUB FRIENDLY']

In [8]:
# GROPU COUNTRIES BY THE CONTINENTS
df['region'] = ['africa' if n in africa_countries 
                else 'africa tournament' if n in africa_tournament
                else 'europa league' if n in eu_countries 
                else 'european tournament' if n in eu_tournament
                else 'asia league' if n in asia_countries
                else 'asia tournament' if n in asia_tournament
                else 'north america league' if n in n_america_countries
                else 'north america tournament' if n in north_america_tournament
                else 'south america league' if n in s_america_countries
                else 'south america tournament' if n in south_america_tournament
                else 'australia league' if n in australia_countries
                else 'australia tournament' if n in australia_tournament
                else 'world' if n in world
                else 0 for n in df['country']]
#df[df['region'] == 0]

In [9]:
df = pd.get_dummies(df, columns=['region'])

In [10]:
def get_dummies_and_merge():
    hosts = pd.get_dummies(df['host'])
    guests = pd.get_dummies(df['guest'])
    # get the same competitors from both columns
    identic_competitors = hosts.columns[hosts.columns.isin(guests.columns)]
    # add values from hosts and guests columns
    dfx = hosts[identic_competitors].values + hosts[identic_competitors].values
    # make DataFrame
    output = pd.DataFrame(dfx, columns=identic_competitors)
    # rest competitors
    rest_comps = hosts[hosts.columns.difference(guests.columns)].join(guests[guests.columns.difference(hosts.columns)])
    # add rest competitors
    output = output.join(rest_comps)
    return output

In [11]:
df_competitors = get_dummies_and_merge()

In [12]:
df_competitors.head()

,07 Vestur Sorvagur,1. SC Feucht,1. SC Sollenau,1.FC Frankfurt,1.FC Monchengladbach,1.FK Svidnik,12 Bingol,12 de Octubre,1625 Liepaja,1874 Northwich,...,Zawisza Rzgow,Zbroslawice,Zell am See,Zenit Tallinn,Znojmo U19,Zorya U19,Zouasia,Zumunta,Zupca,Zwettl
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df = df.join(df_competitors)

In [14]:
df.to_pickle("./clear_df.pkl")

START FROM HERE

In [1]:
import json
import pandas as pd
import numpy as np
df = pd.read_pickle("./clear_df.pkl")

In [2]:
df.head()

,year,month,day,country,tournament,host,guest,ht_score,ft_score,broken,...,Zawisza Rzgow,Zbroslawice,Zell am See,Zenit Tallinn,Znojmo U19,Zorya U19,Zouasia,Zumunta,Zupca,Zwettl
0,2019,2,26,ALGERIA,ALGERIA: LIGUE 2,Harrach,RC Kouba,0:0,0:2,0,...,0,0,0,0,0,0,0,0,0,0
1,2019,2,26,ANGOLA,ANGOLA: GIRABOLA,Kabuscorp,Cubango,1:0,3:2,0,...,0,0,0,0,0,0,0,0,0,0
2,2019,2,26,ARGENTINA,ARGENTINA: SUPERLIGA,Newells Old Boys,San Martin S.J.,1:0,3:0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019,2,26,ARGENTINA,ARGENTINA: PRIMERA B NACIONAL,Temperley,Ferro,0:1,2:2,0,...,0,0,0,0,0,0,0,0,0,0
4,2019,2,26,ARGENTINA,ARGENTINA: PRIMERA B METROPOLITANA,Barracas Central,Dep. Riestra,1:0,1:0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# SPLIT FOR FEATURES AND LABELS
X = df.drop(['ft_score', 'year', 'day', 'month', 'host', 'guest', 
             'ht_score', 'country', 'tournament', 'broken'], axis=1)
y = df['broken']

In [4]:
X.head()

,ht score difference,region_africa,region_africa tournament,region_asia league,region_asia tournament,region_australia league,region_australia tournament,region_europa league,region_european tournament,region_north america league,...,Zawisza Rzgow,Zbroslawice,Zell am See,Zenit Tallinn,Znojmo U19,Zorya U19,Zouasia,Zumunta,Zupca,Zwettl
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=100)
X_pca = pd.DataFrame(pca.fit_transform(X))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression()

In [ ]:
logmodel.fit(X_train, y_train)

In [ ]:
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, predictions)

TEST SECTION

In [ ]:
# bools = []
# for name in df.tournament:
#     if name == 'ITALY: SUPERCOPPA PRIMAVERA':
#         bools.append(True)
#     else:
#         bools.append(False)
# is_lenght = pd.Series(bools)
# df[is_lenght]#[['host', 'guest', 'broken']]#.value_counts()

In [11]:
# month = df['month'].astype('str')

In [ ]:
# output = pd.DataFrame(index=df.index)
# for col, col_data in df.iteritems():
#     if col_data.dtype == object:
#         col_data = pd.get_dummies(col_data, prefix = col)
#     output = output.join(col_data)

In [41]:
comps = pd.DataFrame({'host': ['polska', 'niemcy', 'belgia', 'francja', 'hiszpania', 'polska', 'niemcy', 'estonia', 'walia'],
                     'guest': ['niemcy', 'belgia', 'szwajcaria', 'polska', 'belgia', 'niemcy', 'polska', 'walia', 'estonia']})

In [ ]:
# comps

In [43]:
x = pd.get_dummies(comps, columns=['host'])

In [ ]:
# x

In [19]:
x = pd.get_dummies(comps['host'])

In [20]:
y = pd.get_dummies(comps['guest'])

In [21]:
x_col = x.columns
y_col = y.columns

In [22]:
diff_comps = x_col[x_col.isin(y_col)]

In [23]:
dfx = x[diff_comps].values + y[diff_comps].values

In [24]:
dff = pd.DataFrame(dfx, columns=diff_comps)

In [25]:
s = x[x_col.difference(y_col)].join(y[y_col.difference(x_col)])

In [65]:
s = x[x_col.difference(y_col)]

In [66]:
d = y[y_col.difference(x_col)]

In [27]:
dff = dff.join(s)

In [ ]:
# dff.join(d)